In [ ]:
import os, gc
import zarr
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from typing import Dict
from collections import Counter
from prettytable import PrettyTable

from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.data import ChunkedDataset, LocalDataManager

from l5kit.configs import load_config_data
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from l5kit.visualization import draw_trajectory, draw_reference_trajectory, TARGET_POINTS_COLOR
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from colorama import Fore, Back, Style

print((os.listdir('../input/lyft-motion-prediction-autonomous-vehicles/')))

plt.rc('animation', html='jshtml')

%matplotlib inline

In [ ]:
# animation for scene
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=80)

In [ ]:
import numpy as np
my_arr = np.zeros(3, dtype=[("color", (np.uint8, 3)), ("label", np.bool)])

print(my_arr[0])

In [ ]:
my_arr[0]["color"] = [0, 218, 130]
my_arr[0]["label"] = True
my_arr[1]["color"] = [245, 59, 255]
my_arr[1]["label"] = True
my_arr[1]["color"] = [7, 6, 97]
my_arr[1]["label"] = True

print(my_arr)

In [ ]:
train = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr")
validation = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/validate.zarr")
test = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/test.zarr/")
train.info

In [ ]:
print(f'We have {len(train.agents)} agents, {len(train.scenes)} scenes, {len(train.frames)} frames and {len(train.traffic_light_faces)} traffic light faces in train.zarr.')
print(f'We have {len(validation.agents)} agents, {len(validation.scenes)} scenes, {len(validation.frames)} frames and {len(validation.traffic_light_faces)} traffic light faces in validation.zarr.')
print(f'We have {len(test.agents)} agents, {len(test.scenes)} scenes, {len(test.frames)} frames and {len(test.traffic_light_faces)} traffic light faces in test.zarr.')


In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"

# get configuration yaml
cfg = load_config_data("../input/visualization-config/visualisation_config.yaml")
print(cfg)

In [ ]:
# Raster Parameters
print(f'current raster_param:\n')
for k,v in cfg["raster_params"].items():
    print(f"{k}:{v}")

In [ ]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

In [ ]:
print(dataset_path)

In [ ]:
agents = pd.DataFrame.from_records(zarr_dataset.agents, columns = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities'])
agents.head()

In [ ]:
agents[['centroid_x','centroid_y']] = agents['centroid'].to_list()
agents = agents.drop('centroid', axis=1)
agents_new = agents[["centroid_x", "centroid_y", "extent", "yaw", "velocity", "track_id", "label_probabilities"]]
del agents
agents_new

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
plt.scatter(agents_new['centroid_x'], agents_new['centroid_y'], marker='+')
plt.xlabel('x', fontsize=11); plt.ylabel('y', fontsize=11)
plt.title("Centroids distribution (sample.zarr)")
plt.show()

In [ ]:
agents_new[['extent_x','extent_y', 'extent_z']] = agents_new['extent'].to_list()
agents_new = agents_new.drop('extent', axis=1)
agents = agents_new[["centroid_x", "centroid_y", 'extent_x', 'extent_y', 'extent_z', "yaw", "velocity", "track_id", "label_probabilities"]]
del agents_new
agents

In [ ]:
sns.axes_style("white")

fig, ax = plt.subplots(1,1,figsize=(16,5))

# plt.subplot(1,3,1)
# sns.kdeplot(agents['extent_x'], shade=True, color='red');
# plt.title("Extent_x distribution")

plt.subplot(1,3,1)
sns.kdeplot(agents['extent_y'], shade=True, color='steelblue');
plt.title("Extent_y distribution")

# plt.subplot(1,3,1)
# sns.kdeplot(agents['extent_z'], shade=True, color='green');
# plt.title("Extent_z distribution")

plt.show();

In [ ]:
sns.set_style('whitegrid')

fig, ax = plt.subplots(1,3,figsize=(16,5))
plt.subplot(1,3,1)
plt.scatter(agents['extent_x'], agents['extent_y'], marker='*')
plt.xlabel('ex', fontsize=11); plt.ylabel('ey', fontsize=11)
plt.title("Extent: ex-ey")

plt.subplot(1,3,2)
plt.scatter(agents['extent_y'], agents['extent_z'], marker='*', color="red")
plt.xlabel('ey', fontsize=11); plt.ylabel('ez', fontsize=11)
plt.title("Extent: ey-ez")

plt.subplot(1,3,3)
plt.scatter(agents['extent_z'], agents['extent_x'], marker='*', color="green")
plt.xlabel('ez', fontsize=11); plt.ylabel('ex', fontsize=11)
plt.title("Extent: ez-ex")

plt.show();

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
sns.distplot(agents['yaw'])
plt.title("Yaw Distribution")
plt.show()

In [ ]:
agents[['velocity_x','velocity_y']] = agents['velocity'].to_list()
agents_vel = agents.drop('velocity', axis=1)
agents_v = agents_vel[["centroid_x", "centroid_y", 'extent_x', 'extent_y', 'extent_z', "yaw", "velocity_x", "velocity_y", "track_id", "label_probabilities"]]
del agents
agents_v

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))

with sns.axes_style("whitegrid"):
    sns.scatterplot(x=agents_v["velocity_x"], y=agents_v["velocity_y"], color='k');
    plt.title('Velocity Distribution')

In [ ]:
agents = zarr_dataset.agents
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=["label", "counts"])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

In [ ]:
print(f'{Fore.YELLOW}Total number of agents in sample .zarr files is {Style.RESET_ALL}{len(zarr_dataset.agents)}. {Fore.BLUE}\nAfter summing up the elements in count column we can see we have {Style.RESET_ALL}{(1324481 + 519385 + 6688 + 43182)} {Fore.BLUE}agents in total.')

In [ ]:
scenes = pd.DataFrame.from_records(zarr_dataset.scenes, columns = ['frame_index_interval', 'host', 'start_time', 'end_time'])
scenes.head()

In [ ]:
scenes[['frame_start_index','frame_end_index']] = scenes['frame_index_interval'].to_list()
scenes_new = scenes.drop('frame_index_interval', axis=1)
scenes_new = scenes_new[["frame_start_index", "frame_end_index", 'host', 'start_time', 'end_time']]
del scenes
scenes_new.head()

In [ ]:
f = plt.figure(figsize=(10, 8))
gs = f.add_gridspec(1, 1)

# with sns.axes_style("whitegrid"):
#     ax = f.add_subplot(gs[0,0])
#     sns.scatterplot(scenes_new['frame_start_index'], scenes_new['frame_end_index'])
#     plt.title('Frame Index Interval Distribution')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,0])
    sns.scatterplot(scenes_new['frame_start_index'], scenes_new['frame_end_index'], hue=scenes_new['host'])
    plt.title('Frame Index Interval Distribution (Grouped per host)')
    
f.tight_layout()

In [ ]:
f = plt.figure(figsize=(10, 8))

with sns.axes_style("white"):
    sns.countplot(scenes_new['host']);
    plt.title("Host Count")

In [ ]:
frames = pd.DataFrame.from_records(zarr_dataset.frames, columns = ['timestamp', 'agent_index_interval', 'traffic_light_faces_index_interval', 'ego_translation','ego_rotation'])
frames.head()

In [ ]:
frames[['ego_translation_x', 'ego_translation_y', 'ego_translation_z']] = frames['ego_translation'].to_list()
frames_new = frames.drop('ego_translation', axis=1)
frames_new = frames_new[['timestamp', 'agent_index_interval', 'traffic_light_faces_index_interval',
                         'ego_translation_x', 'ego_translation_y', 'ego_translation_z', 'ego_rotation']]
del frames
frames_new.head()

In [ ]:
f = plt.figure(figsize=(16, 8))
gs = f.add_gridspec(1, 3)

with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,0])
    sns.distplot(frames_new['ego_translation_x'], color='Orange')
    plt.title('Ego Translation Distribution X')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,1])
    sns.distplot(frames_new['ego_translation_y'], color='Red')
    plt.title('Ego Translation Distribution Y')
    
with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[0,2])
    sns.distplot(frames_new['ego_translation_z'], color='Green')
    plt.title('Ego Translation Distribution Z')
    
f.tight_layout()

In [ ]:
f = plt.figure(figsize=(16, 6))
gs = f.add_gridspec(1, 3)

with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,0])
    plt.scatter(frames_new['ego_translation_x'], frames_new['ego_translation_y'],
                    color='darkkhaki', marker='+')
    plt.title('Ego Translation X-Y')
    plt.xlabel('ego_translation_x')
    plt.ylabel('ego_translation_y')
    
with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,1])
    plt.scatter(frames_new['ego_translation_y'], frames_new['ego_translation_z'],
                    color='slateblue', marker='*')
    plt.title('Ego Translation Distribution Y-Z')
    plt.xlabel('ego_translation_y')
    plt.ylabel('ego_translation_z')
    
with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0,2])
    plt.scatter(frames_new['ego_translation_z'], frames_new['ego_translation_x'],
                    color='turquoise', marker='^')
    plt.title('Ego Translation Distribution Z-X')
    plt.xlabel('ego_translation_z')
    plt.ylabel('ego_translation_x')
    
f.tight_layout()

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(16,16))
colors = ['red', 'blue', 'green', 'magenta', 'orange', 'darkblue', 'black', 'cyan', 'darkgreen']
for i in range(0,3):
    for j in range(0,3):
        df = frames_new['ego_rotation'].apply(lambda x: x[i][j])
        plt.subplot(3,3,i * 3 + j + 1)
        sns.distplot(df, hist=False, color = colors[ i * 3 + j  ])
        plt.xlabel(f'r[ {i + 1} ][ {j + 1} ]')
fig.suptitle("Ego rotation angles distribution", size=14)
plt.show()

In [ ]:
traffic_light_faces = pd.DataFrame.from_records(zarr_dataset.tl_faces, columns = ['face_id', 'traffic_light_id', 'traffic_light_face_status'])
traffic_light_faces.head()